In [2]:
!pip install transformers

In [3]:
import numpy as np
from torch.utils.data import Dataset, DataLoader
import re
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer
from transformers import AutoTokenizer, TFAutoModel
import pandas as pd
import tensorflow as tf
#nltk.download('punkt')
#nltk.download('all')

/home/apasalic/anaconda/envs/nlp_env/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-01-29 00:45:14.651392: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [4]:
#FinanceInc/finbert-pretrain
finbert_tokenizer = AutoTokenizer.from_pretrained("FinanceInc/finbert-pretrain", from_pt=True)

In [5]:
# Initialize the lemmatizer
lemmatizer = WordNetLemmatizer()

# Define the stop words
stop_words = set(stopwords.words('english'))

def preprocess_text(text):
    text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)
    text = re.sub(r'\W', ' ', text)
    text = text.lower()
    # Tokenization
    # tokens = nltk.word_tokenize(text)
    # Remove stop words and lemmatize the words
    # tokens = [lemmatizer.lemmatize(token) for token in tokens if token not in stop_words]
    return text

input_text = '\n**User Report**| | | |\n:--|:--|:--|:--\n**Total Submissions**|10|**First Seen In WSB**|2 years ago\n**Total Comments**|332|**Previous Best DD**|\n**Account Age**|3 years|[^scan ^comment ](https://www.reddit.com/message/compose/?to=VisualMod&subject=scan_comment&message=Replace%20this%20text%20with%20a%20comment%20ID%20(which%20looks%20like%20h26cq3k\\)%20to%20have%20the%20bot%20scan%20your%20comment%20and%20correct%20your%20first%20seen%20date.)|[^scan ^submission ](https://www.reddit.com/message/compose/?to=VisualMod&subject=scan_submission&message=Replace%20this%20text%20with%20a%20submission%20ID%20(which%20looks%20like%20h26cq3k\\)%20to%20have%20the%20bot%20scan%20your%20submission%20and%20correct%20your%20first%20seen%20date.)'
preprocessed_text = preprocess_text(input_text)
print(preprocessed_text)

def decontractions(phrase):
    """decontracted takes text and convert contractions into natural form.
     ref: https://stackoverflow.com/questions/19790188/expanding-english-language-contractions-in-python/47091490#47091490"""
    phrase = re.sub(r"won\'t", "will not", phrase)
    phrase = re.sub(r"can\'t", "can not", phrase)
    phrase = re.sub(r"n\'t", " not", phrase)
    phrase = re.sub(r"\'re", " are", phrase)
    phrase = re.sub(r"\'s", " is", phrase)
    phrase = re.sub(r"\'d", " would", phrase)
    phrase = re.sub(r"\'ll", " will", phrase)
    phrase = re.sub(r"\'t", " not", phrase)
    phrase = re.sub(r"\'ve", " have", phrase)
    phrase = re.sub(r"\'m", " am", phrase)


    return phrase

   user report                            total submissions   10   first seen in wsb   2 years ago   total comments   332   previous best dd      account age   3 years   scan  comment     submission   


In [6]:
qna_df_reddit = pd.read_csv('qna_df_reddit.csv')
qna_df_reddit = qna_df_reddit.sample(frac=1).reset_index(drop=True)
qna_df_reddit

,questions,answers,tags
0,my o position am i fuk d i have a severe ad...,yeah you should sell it all now and buy back a...,dividends
1,what s up with market too much red and blood s...,welcome to r dividends if you are new to the...,dividends
2,earnings drop considerably during or after cen...,forward eps instead of actual skip,stockmarket
3,saudi arabia and opec makes surprise 1 millio...,when we really look at the amount petroleum pr...,stocks
4,wework saga cost masayoshi son 11 5 billion a...,wow they sold nvidia at the bottom in jan 201...,stocks
...,...,...,...
41008,investment hindsight lessons from a 50 year ol...,i m going to add one more for the young bucks ...,dividends
41009,getting closer to my second goal of 100 a mon...,how old are you you re likely missing put on ...,dividends
41010,got my dividend portfolio to 10 a month i ...,as a 30 year old i m kicking myself for not be...,dividends
41011,introducing wsb s first ever paper trading co...,beta,wallstreetbets


In [7]:
qna_df_reddit = qna_df_reddit.map(str)


In [8]:
qna_df_reddit['questions'] = qna_df_reddit['questions'].apply(lambda x: np.nan if re.match(r'^\s*$', x) else x)
qna_df_reddit['answers'] = qna_df_reddit['answers'].apply(lambda x: np.nan if re.match(r'^\s*$', x) else x)

qna_df_reddit = qna_df_reddit.dropna().reset_index(drop=True)
qna_df_reddit

,questions,answers,tags
0,my o position am i fuk d i have a severe ad...,yeah you should sell it all now and buy back a...,dividends
1,what s up with market too much red and blood s...,welcome to r dividends if you are new to the...,dividends
2,earnings drop considerably during or after cen...,forward eps instead of actual skip,stockmarket
3,saudi arabia and opec makes surprise 1 millio...,when we really look at the amount petroleum pr...,stocks
4,wework saga cost masayoshi son 11 5 billion a...,wow they sold nvidia at the bottom in jan 201...,stocks
...,...,...,...
40801,investment hindsight lessons from a 50 year ol...,i m going to add one more for the young bucks ...,dividends
40802,getting closer to my second goal of 100 a mon...,how old are you you re likely missing put on ...,dividends
40803,got my dividend portfolio to 10 a month i ...,as a 30 year old i m kicking myself for not be...,dividends
40804,introducing wsb s first ever paper trading co...,beta,wallstreetbets


In [9]:
qna_df_reddit['questions'] = qna_df_reddit['questions'].apply(lambda x: preprocess_text(x))
qna_df_reddit['answers'] = qna_df_reddit['answers'].apply(lambda x: preprocess_text(x))

qna_df_reddit['questions'] = qna_df_reddit['questions'].apply(lambda x: decontractions(x))
qna_df_reddit['answers'] = qna_df_reddit['answers'].apply(lambda x: decontractions(x))

qna_df_reddit['question_tokens'] = qna_df_reddit['questions'].apply(lambda x: finbert_tokenizer.tokenize(x))
qna_df_reddit['answer_tokens'] = qna_df_reddit['answers'].apply(lambda x: finbert_tokenizer.tokenize(x))

qna_df_reddit['question_tokens'] = qna_df_reddit['question_tokens'].apply(lambda x: ['[CLS]'] + x + ['[SEP]'])
qna_df_reddit['answer_tokens'] = qna_df_reddit['answer_tokens'].apply(lambda x: ['[CLS]'] + x + ['[SEP]'])

qna_df_reddit['question_ids'] = qna_df_reddit['question_tokens'].apply(lambda x: finbert_tokenizer.convert_tokens_to_ids(x))
qna_df_reddit['answer_ids'] = qna_df_reddit['answer_tokens'].apply(lambda x: finbert_tokenizer.convert_tokens_to_ids(x))



In [10]:
qna_df_reddit

,questions,answers,tags,question_tokens,answer_tokens,question_ids,answer_ids
0,my o position am i fuk d i have a severe ad...,yeah you should sell it all now and buy back a...,dividends,"[[CLS], my, o, position, am, i, fu, ##k, d, i,...","[[CLS], yeah, you, should, sell, it, all, now,...","[3, 657, 1506, 497, 1198, 44, 9841, 994, 1514,...","[3, 15471, 40, 144, 329, 41, 69, 212, 8, 500, ..."
1,what s up with market too much red and blood s...,welcome to r dividends if you are new to the...,dividends,"[[CLS], what, s, up, with, market, too, much, ...","[[CLS], welcome, to, r, dividends, if, you, ar...","[3, 163, 58, 129, 20, 52, 1727, 406, 2932, 8, ...","[3, 2133, 9, 2859, 751, 60, 40, 21, 56, 9, 6, ..."
2,earnings drop considerably during or after cen...,forward eps instead of actual skip,stockmarket,"[[CLS], earnings, drop, considerably, during, ...","[[CLS], forward, eps, instead, of, actual, ski...","[3, 149, 2535, 6800, 84, 16, 293, 2034, 230, 7...","[3, 663, 249, 3592, 7, 499, 26189, 4]"
3,saudi arabia and opec makes surprise 1 millio...,when we really look at the amount petroleum pr...,stocks,"[[CLS], saudi, arabia, and, opec, makes, surpr...","[[CLS], when, we, really, look, at, the, amoun...","[3, 4807, 6569, 8, 25140, 1221, 5544, 428, 27,...","[3, 181, 13, 392, 368, 28, 6, 224, 3360, 89, 1..."
4,wework saga cost masayoshi son 11 5 billion a...,wow they sold nvidia at the bottom in jan 201...,stocks,"[[CLS], we, ##work, sag, ##a, cost, mas, ##ay,...","[[CLS], wo, ##w, they, sold, nvidia, at, the, ...","[3, 13, 8724, 21481, 363, 121, 13100, 4345, 29...","[3, 9566, 1652, 174, 449, 10627, 28, 6, 2169, ..."
...,...,...,...,...,...,...,...
40801,investment hindsight lessons from a 50 year ol...,i m going to add one more for the young bucks ...,dividends,"[[CLS], investment, hind, ##sight, lesson, ##s...","[[CLS], i, m, going, to, add, one, more, for, ...","[3, 79, 29270, 16560, 23379, 63, 23, 11, 1104,...","[3, 44, 1276, 201, 9, 1122, 137, 59, 14, 6, 66..."
40802,getting closer to my second goal of 100 a mon...,how old are you you re likely missing put on ...,dividends,"[[CLS], getting, closer, to, my, second, goal,...","[[CLS], how, old, are, you, you, re, likely, m...","[3, 1328, 3158, 9, 657, 180, 1937, 7, 1081, 11...","[3, 283, 2664, 21, 40, 40, 482, 419, 11377, 10..."
40803,got my dividend portfolio to 10 a month i ...,as a 30 year old i m kicking myself for not be...,dividends,"[[CLS], got, my, dividend, portfolio, to, 10, ...","[[CLS], as, a, 30, year, old, i, m, kick, ##in...","[3, 831, 657, 699, 318, 9, 685, 11, 804, 44, 1...","[3, 18, 11, 1000, 62, 2664, 44, 1276, 7974, 22..."
40804,introducing wsb s first ever paper trading co...,beta,wallstreetbets,"[[CLS], introducing, ws, ##b, s, first, ever, ...","[[CLS], beta, [SEP]]","[3, 5638, 7765, 974, 58, 78, 3157, 1659, 429, ...","[3, 5457, 4]"


In [11]:
max_question_len = qna_df_reddit['question_ids'].apply(len).max()
max_answer_len = qna_df_reddit['answer_ids'].apply(len).max()
MAX_LEN = max(max_question_len, max_answer_len)

print('Max question length: ', MAX_LEN)

qna_df_reddit['question_ids_pad'] = qna_df_reddit['question_ids'].apply(lambda x: tf.keras.preprocessing.sequence.pad_sequences([x], maxlen=MAX_LEN, padding='post', truncating='post')[0])
qna_df_reddit['answer_ids_pad'] = qna_df_reddit['answer_ids'].apply(lambda x: tf.keras.preprocessing.sequence.pad_sequences([x], maxlen=MAX_LEN, padding='post', truncating='post')[0])

Max question length:  5951


In [12]:
qna_df_reddit['question_attention_mask'] = qna_df_reddit['question_ids_pad'].apply(lambda x: [1 if i != 0 else 0 for i in x])
qna_df_reddit['answer_attention_mask'] = qna_df_reddit['answer_ids_pad'].apply(lambda x: [1 if i != 0 else 0 for i in x])

qna_df_reddit

,questions,answers,tags,question_tokens,answer_tokens,question_ids,answer_ids,question_ids_pad,answer_ids_pad,question_attention_mask,answer_attention_mask
0,my o position am i fuk d i have a severe ad...,yeah you should sell it all now and buy back a...,dividends,"[[CLS], my, o, position, am, i, fu, ##k, d, i,...","[[CLS], yeah, you, should, sell, it, all, now,...","[3, 657, 1506, 497, 1198, 44, 9841, 994, 1514,...","[3, 15471, 40, 144, 329, 41, 69, 212, 8, 500, ...","[3, 657, 1506, 497, 1198, 44, 9841, 994, 1514,...","[3, 15471, 40, 144, 329, 41, 69, 212, 8, 500, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
1,what s up with market too much red and blood s...,welcome to r dividends if you are new to the...,dividends,"[[CLS], what, s, up, with, market, too, much, ...","[[CLS], welcome, to, r, dividends, if, you, ar...","[3, 163, 58, 129, 20, 52, 1727, 406, 2932, 8, ...","[3, 2133, 9, 2859, 751, 60, 40, 21, 56, 9, 6, ...","[3, 163, 58, 129, 20, 52, 1727, 406, 2932, 8, ...","[3, 2133, 9, 2859, 751, 60, 40, 21, 56, 9, 6, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
2,earnings drop considerably during or after cen...,forward eps instead of actual skip,stockmarket,"[[CLS], earnings, drop, considerably, during, ...","[[CLS], forward, eps, instead, of, actual, ski...","[3, 149, 2535, 6800, 84, 16, 293, 2034, 230, 7...","[3, 663, 249, 3592, 7, 499, 26189, 4]","[3, 149, 2535, 6800, 84, 16, 293, 2034, 230, 7...","[3, 663, 249, 3592, 7, 499, 26189, 4, 0, 0, 0,...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, ..."
3,saudi arabia and opec makes surprise 1 millio...,when we really look at the amount petroleum pr...,stocks,"[[CLS], saudi, arabia, and, opec, makes, surpr...","[[CLS], when, we, really, look, at, the, amoun...","[3, 4807, 6569, 8, 25140, 1221, 5544, 428, 27,...","[3, 181, 13, 392, 368, 28, 6, 224, 3360, 89, 1...","[3, 4807, 6569, 8, 25140, 1221, 5544, 428, 27,...","[3, 181, 13, 392, 368, 28, 6, 224, 3360, 89, 1...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
4,wework saga cost masayoshi son 11 5 billion a...,wow they sold nvidia at the bottom in jan 201...,stocks,"[[CLS], we, ##work, sag, ##a, cost, mas, ##ay,...","[[CLS], wo, ##w, they, sold, nvidia, at, the, ...","[3, 13, 8724, 21481, 363, 121, 13100, 4345, 29...","[3, 9566, 1652, 174, 449, 10627, 28, 6, 2169, ...","[3, 13, 8724, 21481, 363, 121, 13100, 4345, 29...","[3, 9566, 1652, 174, 449, 10627, 28, 6, 2169, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
...,...,...,...,...,...,...,...,...,...,...,...
40801,investment hindsight lessons from a 50 year ol...,i m going to add one more for the young bucks ...,dividends,"[[CLS], investment, hind, ##sight, lesson, ##s...","[[CLS], i, m, going, to, add, one, more, for, ...","[3, 79, 29270, 16560, 23379, 63, 23, 11, 1104,...","[3, 44, 1276, 201, 9, 1122, 137, 59, 14, 6, 66...","[3, 79, 29270, 16560, 23379, 63, 23, 11, 1104,...","[3, 44, 1276, 201, 9, 1122, 137, 59, 14, 6, 66...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
40802,getting closer to my second goal of 100 a mon...,how old are you you re likely missing put on ...,dividends,"[[CLS], getting, closer, to, my, second, goal,...","[[CLS], how, old, are, you, you, re, likely, m...","[3, 1328, 3158, 9, 657, 180, 1937, 7, 1081, 11...","[3, 283, 2664, 21, 40, 40, 482, 419, 11377, 10...","[3, 1328, 3158, 9, 657, 180, 1937, 7, 1081, 11...","[3, 283, 2664, 21, 40, 40, 482, 419, 11377, 10...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
40803,got my dividend portfolio to 10 a month i ...,as a 30 year old i m kicking myself for not be...,dividends,"[[CLS], got, my, div

In [13]:
# qna_df_reddit['question_ids_pad'] = qna_df_reddit['question_ids_pad'].apply(lambda x: x.tolist() if not isinstance(x, list) else x)
# qna_df_reddit['answer_ids_pad'] = qna_df_reddit['answer_ids_pad'].apply(lambda x: x.tolist() if not isinstance(x, list) else x)
# qna_df_reddit['question_attention_mask'] = qna_df_reddit['question_attention_mask'].apply(lambda x: x.tolist() if not isinstance(x, list) else x)
# qna_df_reddit['answer_attention_mask'] = qna_df_reddit['answer_attention_mask'].apply(lambda x: x.tolist() if not isinstance(x, list) else x)


In [14]:
from sklearn.model_selection import train_test_split
train, validation = train_test_split(qna_df_reddit, test_size=0.2,random_state=42,shuffle=True)

In [15]:
train

,questions,answers,tags,question_tokens,answer_tokens,question_ids,answer_ids,question_ids_pad,answer_ids_pad,question_attention_mask,answer_attention_mask
27905,insomniac a top videogame developer s leaks r...,chmod x men,stocks,"[[CLS], inso, ##mni, ##ac, a, top, video, ##ga...","[[CLS], ch, ##mod, x, men, [SEP]]","[3, 17981, 25914, 5197, 11, 909, 1780, 28277, ...","[3, 8551, 15618, 3474, 4640, 4]","[3, 17981, 25914, 5197, 11, 909, 1780, 28277, ...","[3, 8551, 15618, 3474, 4640, 4, 0, 0, 0, 0, 0,...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
10888,at approximately what account balance did you ...,it depends on how you look at it i will use o...,dividends,"[[CLS], at, approximately, what, account, bala...","[[CLS], it, depends, on, how, you, look, at, i...","[3, 28, 117, 163, 709, 303, 510, 40, 1016, 9, ...","[3, 41, 1686, 19, 283, 40, 368, 28, 41, 44, 36...","[3, 28, 117, 163, 709, 303, 510, 40, 1016, 9, ...","[3, 41, 1686, 19, 283, 40, 368, 28, 41, 44, 36...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
11436,an honest 5cents earned this gain is unrivaled...,gains are gains most new investors blow their...,stockmarket,"[[CLS], an, honest, 5, ##cent, ##s, earned, th...","[[CLS], gains, are, gains, most, new, investor...","[3, 33, 17848, 898, 8773, 63, 1356, 26, 608, 1...","[3, 596, 21, 596, 242, 56, 311, 12430, 104, 49...","[3, 33, 17848, 898, 8773, 63, 1356, 26, 608, 1...","[3, 596, 21, 596, 242, 56, 311, 12430, 104, 49...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
33060,alibaba co founders buy more than 200 million...,pretty sure china wanting to prop up its stock...,stockmarket,"[[CLS], ali, ##bab, ##a, co, founders, buy, mo...","[[CLS], pretty, sure, china, wanting, to, prop...","[3, 6803, 17284, 363, 3941, 21629, 500, 59, 74...","[3, 1062, 1536, 1308, 17542, 9, 10721, 129, 38...","[3, 6803, 17284, 363, 3941, 21629, 500, 59, 74...","[3, 1062, 1536, 1308, 17542, 9, 10721, 129, 38...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
2752,thanks community 1st of many to come,whoop whoop i still remember my first 12 fr...,dividends,"[[CLS], thanks, community, 1st, of, many, to, ...","[[CLS], who, ##op, who, ##op, i, still, rememb...","[3, 1237, 1968, 8341, 7, 321, 9, 825, 4]","[3, 412, 4567, 412, 4567, 44, 380, 4282, 657, ...","[3, 1237, 1968, 8341, 7, 321, 9, 825, 4, 0, 0,...","[3, 412, 4567, 412, 4567, 44, 380, 4282, 657, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, ..."
...,...,...,...,...,...,...,...,...,...,...,...
6265,bank of england warns u s tech stock valuatio...,they shorted tech stocks to save their economy s,stocks,"[[CLS], bank, of, england, warn, ##s, u, s, te...","[[CLS], they, short, ##ed, tech, stocks, to, s...","[3, 230, 7, 7714, 21798, 63, 4729, 58, 4579, 9...","[3, 174, 1072, 268, 4579, 924, 9, 6177, 104, 1...","[3, 230, 7, 7714, 21798, 63, 4729, 58, 4579, 9...","[3, 174, 1072, 268, 4579, 924, 9, 6177, 104, 1...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, ..."
11284,the warped financial logic fueling america s ...,tl dr the pandemic boarded up so many stor...,finance,"[[CLS], the, war, ##ped, financial, logic, fue...","[[CLS], tl, dr, the, pandemic, board, ##ed, up...","[3, 6, 4661, 7753, 39, 6376, 16729, 723, 58, 1...","[3, 8797, 8836, 6, 16657, 493, 268, 129, 96, 3...","[3, 6, 4661, 7753, 39, 6376, 16729, 723, 58, 1...","[3, 8797, 8836, 6, 16657, 493, 268, 129, 96, 3...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
38158,biden says banking system is safe after sili...,is this political inverse cramer is he tellin...,finance,"[[CLS], bid, ##en, says, banking, system, is, ...","[[CLS], is, this, poli

In [16]:
validation

,questions,answers,tags,question_tokens,answer_tokens,question_ids,answer_ids,question_ids_pad,answer_ids_pad,question_attention_mask,answer_attention_mask
35243,what s the difference between now and the grea...,2008 was primarily a real estate crash because...,dividends,"[[CLS], what, s, the, difference, between, now...","[[CLS], 2008, was, primarily, a, real, estate,...","[3, 163, 58, 6, 1748, 335, 212, 8, 6, 1080, 39...","[3, 519, 35, 106, 11, 373, 558, 6270, 3481, 23...","[3, 163, 58, 6, 1748, 335, 212, 8, 6, 1080, 39...","[3, 519, 35, 106, 11, 373, 558, 6270, 3481, 23...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
30292,calls or puts on tsla,1 stock go up elon future gas station overlo...,wallstreetbets,"[[CLS], calls, or, puts, on, ts, ##la, [SEP]]","[[CLS], 1, stock, go, up, elon, future, gas, s...","[3, 2602, 16, 5960, 19, 5796, 3463, 4]","[3, 428, 93, 478, 129, 29620, 128, 297, 3826, ...","[3, 2602, 16, 5960, 19, 5796, 3463, 4, 0, 0, 0...","[3, 428, 93, 478, 129, 29620, 128, 297, 3826, ...","[1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
10423,dividends in a regular brokerage account many...,i get taxed 30 on all us stock dividends ze...,dividends,"[[CLS], dividends, in, a, regular, brokerage, ...","[[CLS], i, get, taxed, 30, on, all, us, stock,...","[3, 751, 10, 11, 2749, 2267, 709, 321, 1043, 4...","[3, 44, 391, 7939, 1000, 19, 69, 90, 93, 751, ...","[3, 751, 10, 11, 2749, 2267, 709, 321, 1043, 4...","[3, 44, 391, 7939, 1000, 19, 69, 90, 93, 751, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
29321,what ceo scott mcnealy had to say to investors...,there is a reason that someone said idiot bor...,stockmarket,"[[CLS], what, ceo, scott, mcn, ##eal, ##y, had...","[[CLS], there, is, a, reason, that, someone, s...","[3, 163, 2303, 2263, 17679, 24213, 387, 159, 9...","[3, 112, 17, 11, 2040, 15, 8741, 960, 20784, 5...","[3, 163, 2303, 2263, 17679, 24213, 387, 159, 9...","[3, 112, 17, 11, 2040, 15, 8741, 960, 20784, 5...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, ..."
25858,telesat stock surges 50 after satellite inter...,how long were they overpaying their suppliers ...,stockmarket,"[[CLS], telesat, stock, surge, ##s, 50, after,...","[[CLS], how, long, were, they, overp, ##ay, ##...","[3, 22261, 93, 9586, 63, 1104, 293, 2894, 1178...","[3, 283, 637, 57, 174, 22927, 4345, 223, 104, ...","[3, 22261, 93, 9586, 63, 1104, 293, 2894, 1178...","[3, 283, 637, 57, 174, 22927, 4345, 223, 104, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
...,...,...,...,...,...,...,...,...,...,...,...
7845,when you read headlines like this,izbrisano,wallstreetbets,"[[CLS], when, you, read, headlines, like, this...","[[CLS], i, ##zb, ##ris, ##ano, [SEP]]","[3, 181, 40, 1606, 21608, 266, 26, 4]","[3, 44, 21596, 6632, 10225, 4]","[3, 181, 40, 1606, 21608, 266, 26, 4, 0, 0, 0,...","[3, 44, 21596, 6632, 10225, 4, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
6194,absolute legend inverse cramer strikes yet ag...,for someone as idiot as him or clearly bad fa...,stockmarket,"[[CLS], absolute, legend, inverse, cra, ##mer,...","[[CLS], for, someone, as, idi, ##ot, as, him, ...","[3, 2241, 10866, 27154, 6270, 4465, 12206, 143...","[3, 14, 8741, 18, 20784, 5536, 18, 6916, 16, 1...","[3, 2241, 10866, 27154, 6270, 4465, 12206, 143...","[3, 14, 8741, 18, 20784, 5536, 18, 6916, 16, 1...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
7605,deutsche bank kiss of death,user report tota...,wallstreetbets,"[[CLS], deutsche, bank, kis, ##s, of, death, [...","[[CLS], user, report, total, submissions, 5, f...","[3, 1069, 230, 21377, 63, 7, 6539, 4]

In [17]:
# train = pd.DataFrame(train)
# validation = pd.DataFrame(validation)
# train.to_csv('train_data_chatbot.csv',index=False)
# validation.to_csv('validation_data_chatbot.csv',index=False)

In [22]:
# Specify the GPU device
device = '/GPU:0'

# Create a strategy to run on the GPU device
strategy = tf.distribute.OneDeviceStrategy(device)

# Run the model within the strategy's scope
with strategy.scope():
    finbert_model = TFAutoModel.from_pretrained("FinanceInc/finbert-pretrain", from_pt=True)
    finbert_model.summary()


Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already

Model: "tf_bert_model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bert (TFBertMainLayer)      multiple                  109751808 
                                                                 
Total params: 109,751,808
Trainable params: 109,751,808
Non-trainable params: 0
_________________________________________________________________


In [19]:
# from transformers import TFGPT2LMHeadModel

# # Load the GPT-2 model
# gpt2_model = TFGPT2LMHeadModel.from_pretrained("FinanceInc/finbert-pretrain", from_pt=True)

In [20]:
train_data = train #pd.read_csv('train_data_chatbot.csv')

question_ids_pad = train_data['question_ids_pad'].tolist()
answer_ids_pad = train_data['answer_ids_pad'].tolist()
question_attention_mask = train_data['question_attention_mask'].tolist()
answer_attention_mask = train_data['answer_attention_mask'].tolist()


question_ids_pad = tf.convert_to_tensor(question_ids_pad)
answer_ids_pad = tf.convert_to_tensor(answer_ids_pad)
question_attention_mask = tf.convert_to_tensor(question_attention_mask)
answer_attention_mask = tf.convert_to_tensor(answer_attention_mask)


In [23]:
print(question_ids_pad[:2])
question_ids_pad = tf.concat(question_ids_pad, axis=0)
print(question_ids_pad.shape)

#encode data for every 1000 rows
for i in range(0, len(question_ids_pad), 1000):
    encoded_data = finbert_model(question_ids_pad[i:i+999], attention_mask=question_attention_mask)


tf.Tensor(
[[    3 17981 25914 ...     0     0     0]
 [    3    28   117 ...     0     0     0]], shape=(2, 5951), dtype=int32)
(32644, 5951)


2024-01-29 00:59:20.028141: W tensorflow/tsl/framework/bfc_allocator.cc:485] Allocator (mklcpu) ran out of memory trying to allocate 555.79GiB (rounded to 596780371968)requested by op ResourceGather
If the cause is memory fragmentation maybe the environment variable 'TF_GPU_ALLOCATOR=cuda_malloc_async' will improve the situation. 
Current allocation summary follows.
Current allocation summary follows.
2024-01-29 00:59:20.028191: I tensorflow/tsl/framework/bfc_allocator.cc:1039] BFCAllocator dump for mklcpu
2024-01-29 00:59:20.028218: I tensorflow/tsl/framework/bfc_allocator.cc:1046] Bin (256): 	Total Chunks: 0, Chunks in use: 0. 0B allocated for chunks. 0B in use in bin. 0B client-requested in use in bin.
2024-01-29 00:59:20.028228: I tensorflow/tsl/framework/bfc_allocator.cc:1046] Bin (512): 	Total Chunks: 0, Chunks in use: 0. 0B allocated for chunks. 0B in use in bin. 0B client-requested in use in bin.
2024-01-29 00:59:20.028235: I tensorflow/tsl/framework/bfc_allocator.cc:1046] Bin 

ResourceExhaustedError: Exception encountered when calling layer 'embeddings' (type TFBertEmbeddings).

{{function_node __wrapped__ResourceGather_device_/job:localhost/replica:0/task:0/device:CPU:0}} OOM when allocating tensor with shape[32644,5951,768] and type float on /job:localhost/replica:0/task:0/device:CPU:0 by allocator mklcpu [Op:ResourceGather]

Call arguments received by layer 'embeddings' (type TFBertEmbeddings):
  • input_ids=tf.Tensor(shape=(32644, 5951), dtype=int32)
  • position_ids=None
  • token_type_ids=tf.Tensor(shape=(32644, 5951), dtype=int32)
  • inputs_embeds=None
  • past_key_values_length=0
  • training=False

In [ ]:

# Decode the encoded data using the gpt2_model
# decoded_data = gpt2_model.generate(input_ids=encoded_data['last_hidden_state'], attention_mask=answer_attention_mask)